In [21]:
import numpy as np
import pandas as pd
import seaborn as sns
from oneibl import one
from export_funs import trialinfo_to_df
from prior_funcs import fit_sess_psytrack

%matplotlib inline
one = one.ONE(offline=True)

SMALL_SIZE = 16
MEDIUM_SIZE = 20
BIGGER_SIZE = 24

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

session_id = '259927fd-7563-4b03-bc5d-17b4d0fa7a55'
trialsdf = trialinfo_to_df(session_id, maxlen=2.)
# wts, stds = fit_sess_psytrack(session_id, maxlength=2., as_df=True)
# trialsdf = pd.concat((trialsdf, wts['bias']), axis=1)
# bias_next = np.roll(trialsdf['bias'], -1)
# bias_next = pd.Series(bias_next, index=trialsdf['bias'].index)[:-1]
# trialsdf['bias_next'] = bias_next
# spk_times = one.load(session_id, dataset_types=['spikes.times'], offline=True)[0]
# spk_clu = one.load(session_id, dataset_types=['spikes.clusters'], offline=True)[0]

In [22]:
import ipywidgets as iw
from brainbox.plot import peri_event_time_histogram
from brainbox.singlecell import calculate_peths

BINSIZE = 0.005

fitdata = np.load('/home/berk/Documents/Projects/prior-localization/rcos_synthetic_data_multifits_fullkerncov.p', allow_pickle=True)

cells = list(fitdata.keys())
kernels = np.array(['stim', 'feedback', 'wheel'])
combs = fitdata[0].keys()
combnames = {np.array2string(kernels[list(c)]): c for c in combs if type(c) is tuple}


@iw.interact
def plot_fits(cell=cells, combination=combnames.keys()):
    comb = combnames[combination]
    left_rate, fdbk_rate, wheelkern = fitdata[cell]['kernels']
    tstamps_l = np.linspace(0, 0.6, int(0.6 / BINSIZE))
    tstamps_s = np.linspace(0, 0.4, int(0.4 / BINSIZE))
    
    colors = sns.color_palette('viridis', 11)
    ncols = sum(comb) + 1
    size = (16, 5 * (2 + ncols / 3))
    fig, ax = plt.subplots(4, ncols, figsize=size)
    if ncols == 1:
        ax = ax.reshape(-1, 1)
    i = 0
    for j, N in enumerate(x for x in fitdata[cell][comb].keys() if isinstance(x, np.int64)):
        allscores = [trace[-1] for trace in fitdata[cell][comb][N]]
        scorebins = np.linspace(np.min(allscores), np.max(allscores), 10)
        ax[j + 1, -1].hist(allscores)
        ax[j + 1, -1].set_xlim([0, 1])
        ax[j + 1, -1].set_title(f'Mean score of {np.mean(allscores):0.2f}')
    
    if comb[0] is True:
        ax[0, i].plot(tstamps_l, left_rate)
        ax[0, i].set_xlim([0, 0.6])
        ax[0, i].set_title('Original stim on PSTH')
        for j, N in enumerate(x for x in fitdata[cell][comb].keys() if isinstance(x, np.int64)):
            ax[j + 1, i].plot(tstamps_l, left_rate, c='k', label='original', lw=3)
            for trace in fitdata[cell][comb][N]:
                colorind = np.digitize(trace[-1], scorebins)
                ax[j + 1, i].plot(trace[i + 1], color=colors[colorind], alpha=0.25)
            ax[j + 1, i].set_title(f"{N} trials")
            ax[j + 1, i].set_xlim([0, 0.6])
#             ax[j + 1, i].legend()
        i += 1
    if comb[1] is True:
        ax[0, i].plot(tstamps_l, fdbk_rate)
        ax[0, i].set_xlim([0, 0.6])
        ax[0, i].set_title('Original feedback PSTH')
        for j, N in enumerate(x for x in fitdata[cell][comb].keys() if isinstance(x, np.int64)):
            ax[j + 1, i].plot(tstamps_l, fdbk_rate, c='k', label='original', lw=3)
            for trace in fitdata[cell][comb][N]:
                colorind = np.digitize(trace[-1], scorebins)
                ax[j + 1, i].plot(trace[i + 1], color=colors[colorind], alpha=0.25)
            ax[j + 1, i].set_title(f"{N} trials")
            ax[j + 1, i].set_xlim([0, 0.6])
#             ax[j + 1, i].legend()
        i += 1
    if comb[2] is True:
        ax[0, i].plot(tstamps_s - 0.4, wheelkern)
        ax[0, i].set_title('Original wheel kernel')
        for j, N in enumerate(x for x in fitdata[cell][comb].keys() if isinstance(x, np.int64)):
            ax[j + 1, i].plot(tstamps_s - 0.4, wheelkern, c='k', label='original', lw=3)
            for trace in fitdata[cell][comb][N]:
                colorind = np.digitize(trace[-1], scorebins)
                ax[j + 1, i].plot((1 / np.log(1 / 0.02)) * trace[i + 1], color=colors[colorind], alpha=0.25)  # Why the fuck is it always 1/4????
            ax[j + 1, i].set_title(f"{N} trials")
#             ax[j + 1, i].legend()

    ax[0, -1].set_title(r'Distribution of $D^2$')
    ax[0, -1].set_xticks([])
    ax[0, -1].set_yticks([])
    for spine in ax[0, -1].spines:
        ax[0, -1].spines[spine].set_visible(False)
    sns.palplot(colors)
    plt.tight_layout()

interactive(children=(Dropdown(description='cell', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0), Dropdown(…

![Guidos function](guido_brainmapfun.png)

In [23]:
from ibl_pipeline import subject, ephys, histology
from ibl_pipeline.analyses import behavior as behavior_ana
from glob import glob


Connecting gercek@datajoint.internationalbrainlab.org:3306
Connected to https://alyx.internationalbrainlab.org as berk.gercek
Connected to https://alyx.internationalbrainlab.org as berk.gercek


NameError: name 'date' is not defined

In [26]:
regionlabeled = histology.ProbeTrajectory &\
    'insertion_data_source = "Ephys aligned histology track"'
sessions = subject.Subject * subject.SubjectProject * ephys.acquisition.Session *\
    regionlabeled * behavior_ana.SessionTrainingStatus * histology.ClusterBrainRegion
bwm_sess = sessions & 'subject_project = "ibl_neuropixel_brainwide_01"' &\
    'good_enough_for_brainwide_map = 1'

In [28]:
a = bwm_sess.fetch(format='frame')

In [40]:
session_ids = [str(uuid) for uuid in a.session_uuid.unique()]

In [42]:
eid = session_ids[0]
spk_times = one.load(eid, dataset_types=['spikes.times'], offline=True)
spk_clu = one.load(eid, dataset_types=['spikes.clusters'], offline=True)


In [45]:
ephys.AlignedTrialSpikes()

KeyboardInterrupt: 